In [0]:
DELETE FROM workspace.sc_silver.leads_pbs
WHERE converted_contact IS NULL;


-- Keep the first row per (id, created_time) and delete the rest
WITH ranked AS (
  SELECT
      _fivetran_id,
      ROW_NUMBER() OVER (
        PARTITION BY id, created_time
        ORDER BY _fivetran_id      -- any stable tiebreaker works
      ) AS rn
  FROM sc_silver.leads_pbs
)
DELETE FROM sc_silver.leads_pbs
WHERE _fivetran_id IN (SELECT _fivetran_id FROM ranked WHERE rn > 1);


WITH ranked AS (
  SELECT
      _fivetran_id,
      ROW_NUMBER() OVER (
        PARTITION BY id, created_time
        ORDER BY _fivetran_id          -- change this if you prefer a different keeper
      ) AS rn
  FROM sc_silver.contactos_pbs
)
DELETE FROM sc_silver.contactos_pbs
WHERE _fivetran_id IN (SELECT _fivetran_id FROM ranked WHERE rn > 1);


WITH ranked AS (
  SELECT
      _fivetran_id,
      ROW_NUMBER() OVER (
        PARTITION BY id, created_time, id_proposta_realizada
        ORDER BY _fivetran_id
      ) AS rn
  FROM sc_silver.propostas_realizadas
)
DELETE FROM sc_silver.propostas_realizadas
WHERE _fivetran_id IN (SELECT _fivetran_id FROM ranked WHERE rn > 1);


WITH ranked AS (
  SELECT
      _fivetran_id,
      ROW_NUMBER() OVER (
        PARTITION BY id, created_time
        ORDER BY _fivetran_id          -- change if you want a different keeper
      ) AS rn
  FROM sc_silver.deals
)
DELETE FROM sc_silver.deals
WHERE _fivetran_id IN (SELECT _fivetran_id FROM ranked WHERE rn > 1);


WITH ranked AS (
  SELECT
    _fivetran_id,
    ROW_NUMBER() OVER (
      PARTITION BY id, created_time
      ORDER BY _fivetran_id          -- e.g., use updated_at DESC to keep newest
    ) AS rn
  FROM sc_silver.contas_pbs
)
DELETE FROM sc_silver.contas_pbs
WHERE _fivetran_id IN (
  SELECT _fivetran_id FROM ranked WHERE rn > 1
);


WITH ranked AS (
  SELECT
    _fivetran_id,
    ROW_NUMBER() OVER (
      PARTITION BY id, created_time
      ORDER BY _fivetran_id            -- e.g., use updated_at DESC to keep newest
    ) AS rn
  FROM sc_silver.contas_importador_pbs
)
DELETE FROM sc_silver.contas_importador_pbs
WHERE _fivetran_id IN (SELECT _fivetran_id FROM ranked WHERE rn > 1);


WITH ranked AS (
  SELECT
    _fivetran_id,
    ROW_NUMBER() OVER (
      PARTITION BY id, created_time
      ORDER BY _fivetran_id          -- e.g., use updated_at DESC to keep newest
    ) AS rn
  FROM sc_silver.contactos_importador_pbs
)
DELETE FROM sc_silver.contactos_importador_pbs
WHERE _fivetran_id IN (SELECT _fivetran_id FROM ranked WHERE rn > 1);


UPDATE sc_silver.contactos_importador_pbs
SET tipo_viatura = CASE
  -- Build a base token: text before the first [ ... ] OR first quoted item inside [...]
  WHEN lower(
         trim(coalesce(
           CASE WHEN trim(regexp_extract(tipo_viatura, '^[^\\[]+', 0)) <> '' THEN regexp_extract(tipo_viatura, '^[^\\[]+', 0) END,
           CASE WHEN trim(regexp_extract(tipo_viatura, '"([^"]+)"', 1)) <> '' THEN regexp_extract(tipo_viatura, '"([^"]+)"', 1) END
         ))
       ) RLIKE '^nov'            THEN 'Novos'
  WHEN lower(
         trim(coalesce(
           CASE WHEN trim(regexp_extract(tipo_viatura, '^[^\\[]+', 0)) <> '' THEN regexp_extract(tipo_viatura, '^[^\\[]+', 0) END,
           CASE WHEN trim(regexp_extract(tipo_viatura, '"([^"]+)"', 1)) <> '' THEN regexp_extract(tipo_viatura, '"([^"]+)"', 1) END
         ))
       ) RLIKE '^(usad|semi)'    THEN 'Usados'
  WHEN lower(
         trim(coalesce(
           CASE WHEN trim(regexp_extract(tipo_viatura, '^[^\\[]+', 0)) <> '' THEN regexp_extract(tipo_viatura, '^[^\\[]+', 0) END,
           CASE WHEN trim(regexp_extract(tipo_viatura, '"([^"]+)"', 1)) <> '' THEN regexp_extract(tipo_viatura, '"([^"]+)"', 1) END
         ))
       ) RLIKE '^emp'            THEN 'Empresas'

  -- Treat "", [], "null"/"Null"/etc. as NULL
  WHEN lower(trim(tipo_viatura)) IN ('null','none','n/a','na') THEN NULL
  WHEN trim(coalesce(
         CASE WHEN trim(regexp_extract(tipo_viatura, '^[^\\[]+', 0)) <> '' THEN regexp_extract(tipo_viatura, '^[^\\[]+', 0) END,
         CASE WHEN trim(regexp_extract(tipo_viatura, '"([^"]+)"', 1)) <> '' THEN regexp_extract(tipo_viatura, '"([^"]+)"', 1) END
       )) = ''                    THEN NULL

  -- Keep anything else, nicely cased
  ELSE initcap(trim(coalesce(
         CASE WHEN trim(regexp_extract(tipo_viatura, '^[^\\[]+', 0)) <> '' THEN regexp_extract(tipo_viatura, '^[^\\[]+', 0) END,
         CASE WHEN trim(regexp_extract(tipo_viatura, '"([^"]+)"', 1)) <> '' THEN regexp_extract(tipo_viatura, '"([^"]+)"', 1) END
       )))
END;


UPDATE sc_silver.contactos_importador_pbs
SET tipo_viatura = CASE
  WHEN lower(regexp_replace(tipo_viatura, '[^a-z]', '')) RLIKE '^nov'         THEN 'Novos'
  WHEN lower(regexp_replace(tipo_viatura, '[^a-z]', '')) RLIKE '^(usad|semi)' THEN 'Usados'
  WHEN lower(regexp_replace(tipo_viatura, '[^a-z]', '')) RLIKE '^emp'         THEN 'Empresas'
  ELSE tipo_viatura
END;


UPDATE sc_silver.contactos_importador_pbs
SET tipo_viatura = NULL
WHERE regexp_like(tipo_viatura, '(?i)^\\s*\\[?\\s*"?null"?\\s*\\]?\\s*$');

UPDATE sc_silver.contactos_importador_pbs
SET tipo_viatura = NULL
WHERE regexp_like(tipo_viatura, '(?i)^\\s*\\[?"?\\s*null\\s*;?\\s*"?\\]?\\s*$');


UPDATE sc_silver.contas_pbs
SET pais = 'Portugal'
WHERE pais = 'Pt';


UPDATE sc_silver.contratos_financiamento
SET cliente = 'E'
WHERE cliente = 'Eni-p';


UPDATE sc_silver.contratos_financiamento
SET tipo_seguro_credito = 'Tipo_Seguro_Credito'
WHERE tipo_seguro_credito = 'Tipo_Seguro_Cr_dito';


UPDATE sc_silver.deals
SET categoria_negocio = nullif(
  array_join(
    array_sort(
      filter(
        array_distinct(array(
          CASE WHEN regexp_like(categoria_negocio, '(?i)nov')            THEN 'Novos'    END,
          CASE WHEN regexp_like(categoria_negocio, '(?i)usad')           THEN 'Usados'   END,
          CASE WHEN regexp_like(categoria_negocio, '(?i)demo')           THEN 'Demo'     END,
          CASE WHEN regexp_like(categoria_negocio, '(?i)servi(ç|c)o?s?') THEN 'Servicos' END
        )),
        x -> x IS NOT NULL
      )
    ),
    ';'
  ),
  ''
);


UPDATE sc_silver.deals
SET categoria_negocio = CASE
  WHEN regexp_like(categoria_negocio, '(?i)demo')                     THEN 'Demo'
  WHEN regexp_like(categoria_negocio, '(?i)nov')                      THEN 'Novos'
  WHEN regexp_like(categoria_negocio, '(?i)usad')                     THEN 'Usados'
  WHEN regexp_like(categoria_negocio, '(?i)servi(ç|c)o?s?')           THEN NULL
  ELSE NULL
END;


UPDATE sc_silver.historico_de_servicos
SET codigo_postal_concessao = NULL
WHERE regexp_like(coalesce(codigo_postal_concessao, ''), '(?i)^\\s*0+\\s*-\\s*0+\\s*$');


UPDATE sc_silver.historico_de_servicos
SET tipo_de_servico = CASE
  WHEN regexp_replace(lower(coalesce(tipo_de_servico,'')), '[^a-z]', '') RLIKE 'manut|rev|serv'     THEN 'Manutenção'
  WHEN regexp_replace(lower(coalesce(tipo_de_servico,'')), '[^a-z]', '') RLIKE 'repar|arranj|cons'  THEN 'Reparação'
  WHEN regexp_replace(lower(coalesce(tipo_de_servico,'')), '[^a-z]', '') RLIKE 'mecan|eletric|electric|motor'
                                                                                                   THEN 'Mecânica'
  WHEN regexp_replace(lower(coalesce(tipo_de_servico,'')), '[^a-z]', '') RLIKE 'chapa|chapar|chapari'
                                                                                                   THEN 'Chapa'
  WHEN regexp_replace(lower(coalesce(tipo_de_servico,'')), '[^a-z]', '') RLIKE 'pint'               THEN 'Pintura'
  WHEN regexp_replace(lower(coalesce(tipo_de_servico,'')), '[^a-z]', '') RLIKE 'colis'              THEN 'Colisão'
  ELSE 'Manutenção'  -- fallback: garante que não há NULL
END;


UPDATE sc_silver.leads_pbs
SET lead_status_2 = 'Pre-Qualified'
WHERE regexp_replace(lower(coalesce(lead_status_2,'')), '[^a-z]', '') = 'prequalified';


UPDATE sc_silver.propostas_realizadas
SET categoria_proposta = NULL
WHERE regexp_replace(lower(coalesce(categoria_proposta,'')), '[^a-z]', '')
      NOT IN ('novos','usados','demo');


UPDATE sc_silver.propostas_realizadas
SET combustivel = NULL
WHERE regexp_like(coalesce(combustivel,''), '(?i)^\\s*s\\s*/\\s*info\\s*$');


UPDATE sc_silver.propostas_realizadas
SET tipo_cliente = CASE
  WHEN regexp_replace(lower(coalesce(tipo_cliente,'')), '[^a-z]', '') = 'particular'
       THEN 'Particular'
  WHEN regexp_replace(lower(coalesce(tipo_cliente,'')), '[^a-z]', '') = 'empresa'
       THEN 'empresa'
  WHEN regexp_replace(lower(coalesce(tipo_cliente,'')), '[^a-z]', '') = 'eni'
       THEN 'ENI'
  WHEN regexp_replace(lower(coalesce(tipo_cliente,'')), '[^a-z]', '') IN ('rac','xtraflex','extraflex')
       THEN NULL
  ELSE tipo_cliente
END
WHERE regexp_replace(lower(coalesce(tipo_cliente,'')), '[^a-z]', '') IN
      ('particular','empresa','eni','rac','xtraflex','extraflex');


UPDATE sc_silver.propostas_realizadas
SET descricao_pintura = CASE
  WHEN regexp_replace(lower(translate(coalesce(descricao_pintura,''),
         'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
         'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
         '[^a-z]', '') RLIKE 'metaliz.*especial'
    THEN 'Metalizada Especial'

  WHEN regexp_replace(lower(translate(coalesce(descricao_pintura,''),
         'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
         'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
         '[^a-z]', '') RLIKE 'metaliz|^metal$'
    THEN 'Metalizada'

  WHEN regexp_replace(lower(translate(coalesce(descricao_pintura,''),
         'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
         'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
         '[^a-z]', '') RLIKE 'solid'
    THEN 'Sólida'

  ELSE NULL
END;


UPDATE sc_silver.propostas_realizadas
SET estado_do_pedido = trim(
  regexp_replace(
    estado_do_pedido,
    '(?i)^(\\s*Aprovado\\s*)c\\s*/\\s*',   -- "Aprovado  c/  "
    '\\1Com '                               -- → "Aprovado Com "
  )
)
WHERE regexp_like(estado_do_pedido, '(?i)^\\s*Aprovado\\s*c\\s*/');


UPDATE sc_silver.propostas_realizadas
SET estado_do_pedido = 'Aprovado com Valor Diferente'
WHERE regexp_like(
  estado_do_pedido,
  '(?i)^\\s*(valor\\s+diferente|aprovado\\s*(c\\s*/|com)\\s*valor\\s*diferente)\\s*$'
);


UPDATE sc_silver.viaturas
SET combustivel = CASE
  WHEN regexp_replace(lower(translate(coalesce(combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE 'plugin|plug'
    THEN 'Híbrido Plug-in'

  WHEN regexp_replace(lower(translate(coalesce(combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE 'hibrid' AND
       regexp_replace(lower(translate(coalesce(combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE '(gasoleo|diesel)'
    THEN 'Híbrido Gasóleo'

  WHEN (
       regexp_replace(lower(translate(coalesce(combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE 'hibrid' AND
       regexp_replace(lower(translate(coalesce(combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE 'gasolin')
    OR regexp_replace(lower(translate(coalesce(combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE '(electrico|eletrico)gasolina|gasolina(electrico|eletrico)'
    THEN 'Híbrido Gasolina'

  WHEN regexp_replace(lower(translate(coalesce(combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE 'hibrid'
    THEN 'Híbrido'

  WHEN regexp_replace(lower(translate(coalesce(combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE 'gpl' AND
       regexp_replace(lower(translate(coalesce(combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE 'gasolin'
    THEN 'Gasolina/GPL'

  WHEN regexp_replace(lower(translate(coalesce(combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') = 'gpl'
    THEN 'GPL'

  WHEN regexp_replace(lower(translate(coalesce(combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE 'electri'
    THEN 'Elétrico'

  WHEN regexp_replace(lower(translate(coalesce(combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE 'diesel|gasoleo'
    THEN 'Gasóleo'

  WHEN regexp_replace(lower(translate(coalesce(combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE 'gasolin'
    THEN 'Gasolina'

  WHEN regexp_replace(lower(translate(coalesce(combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') IN ('', 'na', 'nd') OR
       regexp_replace(lower(translate(coalesce(combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE 'semespecificar'
    THEN NULL

  ELSE combustivel
END;


UPDATE sc_silver.viaturas
SET tire_source = CASE
  -- várias marcas misturadas -> NULL
  WHEN regexp_replace(lower(translate(coalesce(tire_source,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
       RLIKE '(hank.*kumho|kumho.*hank|hank.*nexen|nexen.*hank|kumho.*nexen|nexen.*kumho)'
    THEN NULL

  -- mapeamentos por marca (inclui typos e sufixos)
  WHEN regexp_replace(lower(translate(coalesce(tire_source,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
       RLIKE 'dunlop'                               THEN 'Michelin'

  WHEN regexp_replace(lower(translate(coalesce(tire_source,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
       RLIKE 'continental'                          THEN 'Continental'

  WHEN regexp_replace(lower(translate(coalesce(tire_source,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
       RLIKE 'michelin|micheline|michelinsport|^mich$|^mich' THEN 'Michelin'

  WHEN regexp_replace(lower(translate(coalesce(tire_source,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
       RLIKE 'hank|han?kook'                        THEN 'Hankook'

  WHEN regexp_replace(lower(translate(coalesce(tire_source,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
       RLIKE 'nexen'                                THEN 'Nexen'

  WHEN regexp_replace(lower(translate(coalesce(tire_source,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
       RLIKE 'pirel'                                THEN 'Pirelli'

  WHEN regexp_replace(lower(translate(coalesce(tire_source,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
       RLIKE 'kumh|kumho'                           THEN 'Kumho'

  WHEN regexp_replace(lower(translate(coalesce(tire_source,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
       RLIKE 'bridgestone'                          THEN 'Bridgestone'

  -- vazios / lixo
  WHEN regexp_replace(lower(translate(coalesce(tire_source,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
       IN ('', 'na', 'nd')                          THEN NULL

  ELSE tire_source
END;


UPDATE sc_silver.viaturas
SET gwms_model_year = NULL
WHERE gwms_model_year = 0
   OR regexp_like(CAST(gwms_model_year AS STRING), '^\\s*0\\s*$');


UPDATE sc_silver.viaturas
SET gwms_engine = NULL
WHERE regexp_like(
  trim(CAST(coalesce(gwms_engine, '') AS STRING)),
  '^0(?:\\.0+)?$'         -- 0, 0.0, 0.000 …
);


UPDATE sc_silver.viaturas
SET model_id = NULL
WHERE regexp_like(
  trim(CAST(coalesce(model_id, '') AS STRING)),
  '^0+(?:\\.0+)?$'     -- "0", "00", "0.0", "0.000", etc.
);


UPDATE sc_silver.viaturas
SET codigo_local = NULL
WHERE regexp_like(coalesce(codigo_local, ''),
                  '(?i)^\\s*n\\s*[/._-]?\\s*d\\s*$');


UPDATE sc_silver.viaturas
SET matricula__vin_ = NULL
WHERE regexp_like(
  trim(CAST(coalesce(`matricula__vin_`, '') AS STRING)),
  '^0+(?:\\.0+)?$'
);


UPDATE sc_silver.viaturas
SET `cilindrada__cm3_` = NULL
WHERE regexp_like(
  trim(CAST(coalesce(`cilindrada__cm3_`, '') AS STRING)),
  '^0+(?:\\.0+)?$'      -- "0", "00", "0.0", "0.000", etc.
);


UPDATE sc_silver.viaturas
SET gwms_combustivel = CASE
  WHEN regexp_replace(lower(translate(coalesce(gwms_combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE 'electr'
    THEN 'Elétrico'

  WHEN regexp_replace(lower(translate(coalesce(gwms_combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE 'diesel|dsl|tdi'
    THEN 'Gasóleo'

  WHEN regexp_replace(lower(translate(coalesce(gwms_combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE 'liquefiedpetrolg|liquifiedpetrolg|lpg|gpl'
    THEN 'GPL'

  WHEN regexp_replace(lower(translate(coalesce(gwms_combustivel,''),
       'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
       'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')),
       '[^a-z]', '') RLIKE 'unleaded|gasolin|petrol|lead'
    THEN 'Gasolina'

  ELSE NULL   -- SOHC/DOHC/MPI/TCI e outros ruídos viram NULL
END;


UPDATE sc_silver.viaturas_demo
SET combustivel = CASE
  -- normalizador base (minúsculas, sem acentos, só letras) aplicado em cada WHEN

  -- híbridos primeiro
  WHEN regexp_replace(lower(translate(coalesce(combustivel,''),
        'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
        'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
       RLIKE '(hibrid|hev)' AND
       regexp_replace(lower(translate(coalesce(combustivel,''),
        'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
        'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
       RLIKE 'gasoleo'
    THEN 'Híbrido Gasóleo'

  WHEN (
       regexp_replace(lower(translate(coalesce(combustivel,''),
        'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
        'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '') RLIKE '(hibrid|hev)' AND
       regexp_replace(lower(translate(coalesce(combustivel,''),
        'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
        'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '') RLIKE 'gasolin'
     )
     OR regexp_replace(lower(translate(coalesce(combustivel,''),
        'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
        'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
        RLIKE '(electrico|eletrico)gasolina|gasolina(electrico|eletrico)'
    THEN 'Híbrido Gasolina'

  -- restantes
  WHEN regexp_replace(lower(translate(coalesce(combustivel,''),
        'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
        'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
       RLIKE 'electr'
    THEN 'Elétrico'

  WHEN regexp_replace(lower(translate(coalesce(combustivel,''),
        'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
        'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
       RLIKE 'gasoleo|diesel'
    THEN 'Gasóleo'

  WHEN regexp_replace(lower(translate(coalesce(combustivel,''),
        'áàâãäéèêëíìîïóòôõöúùûüçÁÀÂÃÄÉÈÊËÍÌÎÏÓÒÔÕÖÚÙÛÜÇ',
        'aaaaaeeeeiiiiooooouuuucAAAAAEEEEIIIIOOOOOUUUUC')), '[^a-z]', '')
       RLIKE 'gasolin'
    THEN 'Gasolina'

  ELSE combustivel
END;


UPDATE sc_silver.viaturas_demo
SET combustivel = 'Híbrido Gasolina'
WHERE regexp_like(coalesce(combustivel,''), '(?i)^\\s*hev\\s*$');


UPDATE sc_silver.viaturas_demo
SET viaturas_demo_owner_name = NULL
WHERE
  regexp_like(CAST(coalesce(viaturas_demo_owner_name,'') AS STRING),
              '(?i)^\\s*admin\\s*hpk\\s*$')
  OR regexp_replace(lower(coalesce(viaturas_demo_owner_name,'')), '[^a-z]', '') = 'adminhpk';


UPDATE sc_silver.campanha_vouchers
SET oferta_monetaria = NULL
WHERE try_cast(
        replace(regexp_replace(coalesce(oferta_monetaria,''), '[^0-9,.-]', ''), ',', '.'
      ) AS DOUBLE) = 0.0;


UPDATE sc_silver.campanha_vouchers
SET oferta_percentual = NULL
WHERE try_cast(
        replace(regexp_replace(coalesce(oferta_percentual,''), '[^0-9,.-]', ''), ',', '.'
      ) AS DOUBLE) = 0.0;


UPDATE sc_silver.campanhas_tecnicas
SET warranty_start_date = NULL
WHERE warranty_start_date = DATE '9999-12-31';


UPDATE sc_silver.contactos_importador_pbs
SET id_contactos = NULL
WHERE regexp_like(coalesce(id_contactos, ''), '(?i)^\s*null\s*$');


UPDATE sc_silver.contratos_financiamento
SET capital_em_divida = NULL
WHERE regexp_like(
        -- keep only digits and decimal/thousand separators
        regexp_replace(coalesce(capital_em_divida, ''), '[^0-9,.-]', ''),
        -- match 0 / 00 / 0.00 / 0,000 etc., but NOT 0,01
        '^[+-]?0+(?:[.,]0+)?$'
      );


UPDATE sc_silver.contratos_financiamento
SET prazo = NULL
WHERE regexp_like(
  trim(CAST(coalesce(prazo, '') AS STRING)),
  '^[+-]?0+(?:\\.0+)?$'    -- 0, 00, 0.0, 0.00, etc.
);


UPDATE sc_silver.contratos_financiamento
SET odometer = NULL
WHERE regexp_like(
  replace(regexp_replace(CAST(coalesce(odometer,'') AS STRING), '[^0-9,.-]', ''), ',', '.'),
  '^[+-]?0+(?:\\.0+)?$'
);


UPDATE sc_silver.contratos_financiamento
SET preco_maximo_de_compra = NULL
WHERE try_cast(
        replace(regexp_replace(coalesce(preco_maximo_de_compra,''), '[^0-9,.-]', ''), ',', '.'
      ) AS DOUBLE) = 0.0;


UPDATE sc_silver.contratos_financiamento
SET retail = NULL
WHERE try_cast(
        replace(regexp_replace(coalesce(CAST(retail AS STRING), ''), '[^0-9,.-]', ''), ',', '.'
      ) AS DOUBLE) = 0.0;

  
UPDATE sc_silver.contratos_financiamento
SET valor_final = NULL
WHERE try_cast(
        replace(regexp_replace(coalesce(valor_final,''), '[^0-9,.-]', ''), ',', '.'
      ) AS DOUBLE) = 0.0;

  
UPDATE sc_silver.deals
SET sc_silver.deals.stage = 'Venda - Aprovado'
WHERE regexp_like(
  coalesce(sc_silver.deals.stage, ''),
  '(?i)^\s*venda\s*[-–]?\s*aprovado\s*$'   -- matches "Venda-aprovado", "Venda – aprovado", "Venda aprovado"
);


UPDATE sc_silver.deals
SET age_in_days = NULL
WHERE age_in_days = 0;


UPDATE sc_silver.deals
SET motivo_da_perda = NULL
WHERE regexp_like(coalesce(motivo_da_perda, ''), '(?i)^\s*null\s*$');


UPDATE sc_silver.historico_de_servicos
SET canal_de_venda = NULL
WHERE regexp_like(
  trim(CAST(coalesce(canal_de_venda, '') AS STRING)),
  '^[+-]?0+(?:[.,]0+)?$'   -- matches "0", "0.0", "0,0", "00", etc.
);


UPDATE sc_silver.historico_de_servicos
SET total_pecas = NULL
WHERE COALESCE(
        try_cast(total_pecas AS DOUBLE),                                                  -- if numeric
        try_cast(                                                                          -- if string like "€ 0,00"
          replace(regexp_replace(CAST(coalesce(total_pecas,'') AS STRING), '[^0-9,.-]', ''),
                  ',', '.') AS DOUBLE)
      ) = 0.0;


UPDATE sc_silver.historico_de_servicos
SET kms = NULL
WHERE regexp_like(
  trim(CAST(coalesce(kms, '') AS STRING)),
  '^[+-]?0+(?:[.,]0+)?$');    -- 0, 00, 0.0, 0,0, etc.


UPDATE sc_silver.leads_pbs
SET lead_status = 'Pre Qualified'
WHERE regexp_replace(
        lower(translate(coalesce(lead_status,''),
                        'áàâãäéèêëíìîïóòôõöúùûüç',
                        'aaaaaeeeeiiiiooooouuuuc')),
        '[^a-z]', ''
      ) = 'emqualificacao';


UPDATE sc_silver.leads_pbs
SET consentimento = 'Autoriza'
WHERE regexp_like(coalesce(consentimento,''), '(?i)^\\s*sim\\s*$');


UPDATE sc_silver.leads_pbs
SET concessao = NULL
WHERE regexp_like(coalesce(concessao, ''), '^\s*[-–—]\s*$');


UPDATE sc_silver.propostas_realizadas
SET gestor_area = NULLIF(
  trim(
    regexp_replace(
      regexp_replace(coalesce(gestor_area,''), '(?i)[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\\.[A-Za-z]{2,}', ''),  -- remove email
      '^\\s*(.*?)(?:\\s*[-–—].*)?$', '\\1'                                                                -- keep text before first dash, if any
    )
  ),
  ''  -- if nothing remains, set to NULL
)
WHERE gestor_area IS NOT NULL
  AND regexp_like(gestor_area, '(?i)@');   -- only rows that contain an email


UPDATE sc_silver.propostas_realizadas
SET gestor_area = NULL
WHERE regexp_like(CAST(coalesce(gestor_area,'') AS STRING), '^\s*1\s*$');


UPDATE sc_silver.propostas_realizadas
SET tylacode = NULL
WHERE regexp_like(CAST(coalesce(tylacode,'') AS STRING), '(?i)^\s*null\s*$');


UPDATE sc_silver.propostas_realizadas
SET valor_do_apoio_pedido = NULL
WHERE
  -- Check if the decimal value is zero (accounting for 2 decimal places)
  valor_do_apoio_pedido = 0.00;


UPDATE sc_silver.propostas_realizadas
SET s_g_p_u_ = NULL
WHERE COALESCE(
        try_cast(s_g_p_u_ AS DOUBLE),                                                  -- numeric already
        try_cast(                                                                      -- or a string like "€ 0,00"
          replace(regexp_replace(CAST(coalesce(s_g_p_u_,'') AS STRING), '[^0-9,.-]', ''),
                  ',', '.') AS DOUBLE)
      ) = 0.0;

  
UPDATE sc_silver.deals
SET `probability____` = NULL
WHERE `probability____` = 0.00;


UPDATE sc_silver.deals
SET `n__matriculas_por_associar` = NULL
WHERE `n__matriculas_por_associar` = 0;

  
UPDATE sc_silver.deals
SET `n__matriculas_associadas` = NULL
WHERE `n__matriculas_associadas` = 0;

  
UPDATE sc_silver.deals
SET `n__matriculas_por_associar` = NULL
WHERE `n__matriculas_por_associar` < 0;


UPDATE sc_silver.viaturas
SET viatura_owner_name = NULL
WHERE regexp_like(
  CAST(coalesce(viatura_owner_name,'') AS STRING),
  '(?i)^\\s*admin(?:\\s*[-_]*\\s*hpk)?\\s*$'
);